# BSC-HGP - Assignment 2 


| Course 	           | BSc                   |
|  ------------- :     |:-------------        :|  
| Stage / Year         | 1                     |
| Module 	           | HCI & GUI Programming | 
| Semester             |	1                  |
| Assignment           |	Assignment 02      | 
| Date of Title Issue  |	23nd Oct 2018      |
| Assignment Deadline  |	12th Nov 2018 23:55|
| Assignment %         | 15 %                  |
| Assignment Submission| Upload via Moodle      |

Author: Alex Cronin

# Lisa Lewitanski - 2982190


## Design Decisions: 
This program is a Paint application.

It allows the user to create a drawing with some tools.
This application is reactive, when the user change the configuration the pen will get the new configuration and display
the line with it. 

I have created a menu bar containing three menus.
First I have placed the File option. It contains a submenu which allows the user to open, save and clear actions.
Then I have placed the Help option. It contains a submenu which allows the help action.
To finish, I have placed an Exit menu. This menu contains a submenu which allows the exit action.
I have designed my menu like this because it's the usual way to do it.
Thus, the user will instinctively find the correct button. 


I have decided to display the drawing at the left of the window and the drawing settings at the right of the screen to 
have a clearly separate the two parts.

All settings widget are placed vertically in to groupbox. In this way, all widget have the same patterns to avoid
disturbing the user. 
The cap and the join style are placed in to checkbox because it's easy to choose the good configuration without 
a lot of text.   
Then the user can choose to select the pen style and the pen width by clicking on the configuration's image.
I have decided to create an image button because the user can easily choose what he wants.
I also placed a button to choose the color. Like this when the user clicks on the button a box will appear and 
the user will be able to choose the new color.

The color's application is white and beige because there are basic, clear and neutral. 

## Solution


In [3]:
from PyQt5 import QtGui, QtCore
from PyQt5.QtWidgets import QApplication, QMainWindow, QAction, QFileDialog, QWidget, QVBoxLayout, QLCDNumber, QSlider, QMessageBox, QLabel
from PyQt5.QtGui import QIcon, QImage, QPainter, QPen, QPixmap, QLinearGradient, QPainterPath
import sys
import os
from PyQt5.QtCore import Qt, QPoint, QRect, pyqtSlot
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtCore import QDate


class Window(QMainWindow):
    """
    Window is the class which create the paint widget.
    It herits of QMainWindow.
    """

    def __init__(self):
        """
        Init will initialize the window's size and the GridLayout for the settings.
        It also initialize all widgets contained in the GridLayout.

        :param drawing: The Qpaint classes
        """

        super().__init__()

        # Window's size
        top = 400
        left = 400
        self.width = 1000
        self.height = 1010

        # Initialization of the Window with the title, the size and the icon
        self.setWindowTitle("Paint Application")
        self.setGeometry(top, left, self.width, self.height)
        self.setWindowIcon(QIcon("./icons/paint-brush.png"))

        # Image settings (by default)
        self.image = QImage(self.size(), QImage.Format_RGB32)
        self.image = self.image.scaled(self.width, self.height)
        self.image.fill(Qt.white)

        # Draw settings (default)
        self.drawing = False
        self.brushSize = 3
        self.brushColor = Qt.black
        self.pen_style = Qt.SolidLine
        self.pen_cap = Qt.RoundCap
        self.pen_join = Qt.RoundJoin

        # Reference to last point recorded by mouse
        self.lastPoint = QPoint()

        # menus
        mainMenu = self.menuBar()
        self.fileMenu = mainMenu.addMenu(" File")
        self.helpMenu = mainMenu.addMenu(" Help")
        self.exitMenu = mainMenu.addMenu(" Exit")
        self.open_setting()
        self.save_setting()
        self.clear_setting()
        self.help_setting()
        self.exit_setting()

    def mousePressEvent(self, event):
        """
        This method is an event handler.
        If the left button is clicked, it set the draw mode to true and save the last point.

        :param event: The event
        :return: Nothing
        """
        if event.button() == Qt.LeftButton:
            self.drawing = True
            self.lastPoint = event.pos()

    def mouseMoveEvent(self, event):
        """
        This method is an event handler.

        It declare and initialize the style pen with the following values:
                       the color, the width, style, cap style and the join style
        Then it draw at the last point with it and update the window.

        :param event: The event
        :return: Nothing
        """
        if (event.buttons() & Qt.LeftButton) & self.drawing:
            # Object which allows drawing to take place on an image
            painter = QPainter(self.image)
            painter.setPen(QPen(self.brushColor, self.brushSize, self.pen_style, self.pen_cap, self.pen_join))
            painter.drawLine(self.lastPoint, event.pos())
            self.lastPoint = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        """
        If the left button is clicked this method set the draw mode to false to stop the pen.

        :param event: The event
        :return: Nothing
        """
        if event.button == Qt.LeftButton:
            self.drawing = False

    def paintEvent(self, event):
        """
        This method will initialize the QPainter and will draw the rectangular portion source of the given image
        with its origin at the given point.

        :param event: The event
        :return: Nothing
        """
        canvasPainter = QPainter(self)
        canvasPainter.drawImage(self.rect(), self.image, self.image.rect())

    def open_setting(self):
        """
        This method add the open settings to the menu bar with an icon.
        Then it assign a shortcut to this setting. (Ctrl+O)
        If the user clicks on this setting, the open method will be called.

        :return: Nothing
        """

        openAction = QAction(QIcon("./icons/open.png"), "Open", self)
        openAction.setShortcut("Ctrl+O")
        self.fileMenu.addAction(openAction)
        openAction.triggered.connect(self.open)

    def save_setting(self):
        """
        This method add the save option to the menu bar with an icon.
        Then it assign a shortcut to this setting. (Ctrl+S)
        If the user clicks on this setting, the save method will be called.

        :return: Nothing
        """

        saveAction = QAction(QIcon("./icons/save.png"), "Save", self)
        saveAction.setShortcut("Ctrl+S")
        self.fileMenu.addAction(saveAction)
        saveAction.triggered.connect(self.save)

    def clear_setting(self):
        """
        This method add the clear setting to the menu bar with an icon.
        Then it assign a shortcut to this setting. (Ctrl+C)
        If the user clicks on this setting, the clear method will be called.

        :return: Nothing
        """

        clearAction = QAction(QIcon("./icons/clear.png"), "Clear", self)
        clearAction.setShortcut("Ctrl+C")
        self.fileMenu.addAction(clearAction)
        clearAction.triggered.connect(self.clear)

    def help_setting(self):
        """
        This method add the help setting to the menu bar with an icon.
        Then it assign a shortcut to this setting. (Ctrl+H)
        If the user clicks on this setting, the help method will be called.

        :return: Nothing
        """

        helpAction = QAction(QIcon("./icons/help.png"), "Help", self)
        helpAction.setShortcut("Ctrl+H")
        self.helpMenu.addAction(helpAction)
        helpAction.triggered.connect(self.help)

    def exit_setting(self):
        """
        This method add the exit setting to the menu bar with an icon.
        Then it assign a shortcut to this setting. (Ctrl+E)
        If the user clicks on this setting, the exit method will be called.

        :return: Nothing
        """

        exitAction = QAction(QIcon("./icons/exit.png"), "Exit", self)
        exitAction.setShortcut("Ctrl+E")
        self.exitMenu.addAction(exitAction)
        exitAction.triggered.connect(self.exit)

    def open(self):
        """
        This method allows the user to open an image to draw on it.
        First the user choose the file, then it display the image with the window's size.
        :return:
        """
        options = QFileDialog.Options()
        options |= QFileDialog.DontUseNativeDialog
        fileName, _ = QFileDialog.getOpenFileName(self, "QFileDialog.getOpenFileName()", "",
                                                  "All Files (*);;Python Files (*.py)", options=options)
        if fileName:
            self.image = QImage(fileName)
            self.image = self.image.scaled(self.width, self.height)
            self.show()

    def save(self):
        """
        This method allows the user to save his drawing.
        If the file is empty the function returns.
        Else it save the drawing with the path selected.
        :return:
        """
        filePath, _ = QFileDialog.getSaveFileName(self, "Save Image", "",  "PNG(*.png);;JPG(*.jpg *.jpeg);;All Files (*.*)")
        if filePath == "":
            return
        self.image.save(filePath)

    def clear(self):
        """
        This method allows the user to clear his drawing.

        :return: Nothing
        """
        self.image.fill(Qt.white)
        self.update()

    def help(self):
        """
        This method display a QMessageBox. This box helps the user to know what the application
        exactyly doing.

        :return: Nothing
        """
        title = "Help"
        message = "First you can choose the Cap Style by selecting the good one.\n" \
                  "Then you can choose the Join Style by selecting the good one.\n" \
                  "You can choose your Pen Style clicking on the image\n" \
                  "You can choose your Pen Width clicking on the image\n" \
                  "You can choose a color with clicking on the button. After just select the color ends up with 'Ok'"
        QMessageBox.about(self, title, message)
        self.show()

    def exit(self):
        """
        This method allows the user to quit the application.
        In a first time, an QMessageBox appear and the user can choose if he still want to quit the application.
        If he choose 'yes', the application will be closed.

        :return: Nothing
        """
        reply = QMessageBox.question(self, 'Message',
                                     "Are you sure to quit?", QMessageBox.Yes |
                                     QMessageBox.No, QMessageBox.No)
        if reply == QMessageBox.Yes:
            sys.exit()


class Settings(QDialog):
    """
    Settings is the class to create all settings widget and set pen values.
    It herits of QDialod.
    """

    def __init__(self, drawing):
        """
        Init will initialize the window's size and the GridLayout of the settings.
        It's also initialize all widgets contained in the GridLayout.


        :param drawing: The Qpaint class
        """
        super().__init__()

        # Initialization of the window's size
        top = 400
        left = 400
        width = 800
        height = 600

        # Declaration of the QPaint class to access to the pen.
        self.draw = drawing

        # Initialization of each pen state
        self.cap_layout_value = 1
        self.join_layout_value = 1
        self.path_layout_value = 1

        self.cs = QLabel('Cap Style')
        self.setWindowTitle("Paint Application")

        # Initialization of each pen style's images
        self.style1 = QPixmap("style1.jpg")
        self.style2 = QPixmap("style2.jpg")
        self.style3 = QPixmap("style3.jpg")
        self.style4 = QPixmap("style4.jpg")
        self.style5 = QPixmap("style5.jpg")

        # Declaration of each widgets contains in the GridLayout
        self.brush_colour = QGroupBox("Pen Color")
        self.brush_thickness = QGroupBox("Pen Width")
        self.brush_line_type = QGroupBox("Pen Style")
        self.brush_cap_type = QGroupBox("Cap Style")
        self.brush_join_type = QGroupBox("Join Style")
        self.brush_path_composed = QGroupBox("Path composed of")

        # Initialization of each widgets contains in the GridLayout
        self.cap_style_layout()
        self.join_style_layout()
        self.pen_style_layout()
        self.pen_width_layout()
        self.change_color_layout()
        self.path_composed_layout()

        # Declaration and initialization of the GridLayout
        grid = QGridLayout()
        grid.addWidget(self.brush_cap_type, 0, 1)
        grid.addWidget(self.brush_join_type, 1, 1)
        grid.addWidget(self.brush_line_type, 2, 1)
        grid.addWidget(self.brush_thickness, 3, 1)
        grid.addWidget(self.brush_colour, 4, 1)
        self.setLayout(grid)

    def cap_style_layout(self):
        """
        cap_style_layout method create an QVBoxLayout in a QGroupBox.
        In this way I can create three RadioButtons for each Cap Style (flat, square and round cap).
        In a first time the cap style is initialized to flat cap.
        If the user decide to change the cap style and click on the button, the 'btncap' method will be called with the
        user's selection in parameter.

        :return: Nothing
        """
        self.cap_layout = QVBoxLayout()
        self.cap_layout.addStretch(1)

        # Initialization of each QRadioButton
        self.cap1 = QRadioButton("Flat Cap")
        self.cap2 = QRadioButton("Square Cap")
        self.cap3 = QRadioButton("Round Cap")

        self.cap1.setChecked(True)
        self.cap1.toggled.connect(lambda: self.btncap(self.cap1))
        self.cap_layout.addWidget(self.cap1)

        self.cap2.toggled.connect(lambda: self.btncap(self.cap2))
        self.cap_layout.addWidget(self.cap2)

        self.cap3.toggled.connect(lambda: self.btncap(self.cap3))
        self.cap_layout.addWidget(self.cap3)

        self.cap_layout.addStretch(1)
        self.brush_cap_type.setLayout(self.cap_layout)

    def checkWidth(self, pen, numpen, icon):
        """
        The pen's layout will be initialized with the icon and the icon's size.
        Then the method call the method to assign the new width to the pen.

        :param pen: The pen's width layout.
        :param numpen: The pen's number 1px = 1, 2px = 2 etc.
        :param icon: The image's icon set to each width pen.
        :return: The pen's width layout.
        """
        pen.setChecked(True)
        pen.setFlat(True)
        if numpen == 1:
            pen.clicked.connect(self.btnwidth1px)
        elif numpen == 2:
            pen.clicked.connect(self.btnwidth3px)
        elif numpen == 3:
            pen.clicked.connect(self.btnwidth5px)
        else:
            pen.clicked.connect(self.btnwidth8px)
        pen.setStyleSheet('border:none')
        pen.setIcon(icon)
        pen.setIconSize(QtCore.QSize(100, 30))
        return pen

    def pen_width_layout(self):
        """
        pen_width_layout method create an QVBoxLayout in a QGroupBox.
        Four image's button widgets will be created with the checkWidth method.
        If the user choose the pen's width it also set the value to the pen.

        :return: Nothing
        """
        self.pen_width = QVBoxLayout()
        self.pen_width.addStretch(1)

        self.pen_width1 = self.checkWidth(QPushButton(" 1px"), 1, QIcon("1px.png"))
        self.pen_width2 = self.checkWidth(QPushButton(" 3px"), 2, QIcon("3px.png"))
        self.pen_width3 = self.checkWidth(QPushButton(" 5px"), 3, QIcon("5px.png"))
        self.pen_width4 = self.checkWidth(QPushButton(" 8px"), 4, QIcon("8px.png"))

        self.pen_width.addWidget(self.pen_width1)
        self.pen_width.addWidget(self.pen_width2)
        self.pen_width.addWidget(self.pen_width3)
        self.pen_width.addWidget(self.pen_width4)

        self.pen_width.addStretch(1)
        self.brush_thickness.setLayout(self.pen_width)

    def change_color_layout(self):
        """
        change_color_layout method create an QVBoxLayout in a QGroupBox.
        This method creates a button to allow the user to change the pen's color.
        If the user click on choose an other color the on_click method will be called.

        :return: Nothing
        """
        color = QVBoxLayout()
        color.addStretch(1)

        # Button's initialization
        button = QPushButton('Choose your color', self)
        button.setToolTip('Choose your color')
        # Set
        button.move(10, 10)
        button.clicked.connect(self.on_click)

        # Add the color's button to the color layout.
        color.addWidget(button)

        color.addStretch(1)
        self.brush_colour.setLayout(color)

    @pyqtSlot()
    def on_click(self):
        """
        This method call the openColorDialog method.

        :return: Nothing
        """
        self.openColorDialog()

    def openColorDialog(self):
        """
        This method creates a QColorDialog to choose the color.
        If the color is valid, the pen's color will be set to this color.

        :return: Nothing
        """
        color = QColorDialog.getColor()

        if color.isValid():
            self.draw.brushColor = color
            self.show()

    def join_style_layout(self):
        """
        join_style_layout method create an QVBoxLayout in a QGroupBox.
        In this way I can create three RadioButton for each Cap Style (bevel, miter and round join).
        In a first time the join style is initialized to bevel join.
        If the user decide to change the join style and click on the button, the 'btnjoin' method will be called with the
        user's selection in parameter.

        :return: Nothing
        """

        self.join_layout = QVBoxLayout()
        self.join_layout.addStretch(1)

        self.join1 = QRadioButton("Bevel Join")
        self.join1.setChecked(True)
        self.join1.toggled.connect(lambda: self.btnjoin(self.join1))
        self.join_layout.addWidget(self.join1)
        self.join2 = QRadioButton("Miter Join")

        self.join2.toggled.connect(lambda: self.btnjoin(self.join2))
        self.join_layout.addWidget(self.join2)
        self.join3 = QRadioButton("Round Join")

        self.join3.toggled.connect(lambda: self.btnjoin(self.join3))
        self.join_layout.addWidget(self.join3)
        self.join_layout.addStretch(1)
        self.brush_join_type.setLayout(self.join_layout)

    def path_composed_layout(self):
        self.path_layout = QVBoxLayout()
        self.path_layout.addStretch(1)

        self.path1 = QRadioButton("Curves")
        self.path1.setChecked(True)
        self.path1.toggled.connect(lambda: self.btnpath(self.path1))
        self.path_layout.addWidget(self.path1)

        self.path2 = QRadioButton("Lines")
        self.path2.toggled.connect(lambda: self.btnpath(self.path2))
        self.path_layout.addWidget(self.path2)

        self.path_layout.addStretch(1)
        self.brush_path_composed.setLayout(self.path_layout)

    def checkStyle(self, pen, numStyle, icon):
        """
        The pen's layout will be initialized with the icon and the icon's size.
        Then the method call the method to assign the new style to the pen.

        :param pen: The pen style layout.
        :param numStyle: The pen's number 1px = 1, 2px = 2 etc.
        :param icon: The image's icon set to each width pen.
        :return: The pen style layout.
        """
        pen.setChecked(True)
        pen.setFlat(True)
        if numStyle == 1:
            pen.clicked.connect(self.btnSolidLine)
        elif numStyle == 2:
            pen.clicked.connect(self.btnDashLine)
        elif numStyle == 3:
            pen.clicked.connect(self.btnDotLine)
        elif numStyle == 4:
            pen.clicked.connect(self.btndashDotLine)
        elif numStyle == 5:
            pen.clicked.connect(self.btnDashDotDotLine)
        else:
            pen.clicked.connect(self.btnCustomDashLine)
        pen.setStyleSheet('border:none')
        pen.setIcon(icon)
        pen.setIconSize(QtCore.QSize(100, 30))
        return pen

    def pen_style_layout(self):
        """
        pen_style_layout method creates an QVBoxLayout in a QGroupBox.
        Six image's button widgets will be created with the checkStyle method.
        If the user choose the pen's style it also set the style to the pen.

        :return: Nothing
        """
        self.pen_style = QVBoxLayout()
        self.pen_style.addStretch(1)

        self.pen_style1 = self.checkStyle(QPushButton(""), 1, QIcon("line1.png"))
        self.pen_style2 = self.checkStyle(QPushButton(""), 2, QIcon("line2.png"))
        self.pen_style3 = self.checkStyle(QPushButton(""), 3, QIcon("line3.png"))
        self.pen_style4 = self.checkStyle(QPushButton(""), 4, QIcon("line4.png"))
        self.pen_style5 = self.checkStyle(QPushButton(""), 5, QIcon("line5.png"))
        self.pen_style6 = self.checkStyle(QPushButton(""), 6, QIcon("custom_style.png"))

        self.pen_style.addWidget(self.pen_style1)
        self.pen_style.addWidget(self.pen_style2)
        self.pen_style.addWidget(self.pen_style3)
        self.pen_style.addWidget(self.pen_style4)
        self.pen_style.addWidget(self.pen_style5)
        self.pen_style.addWidget(self.pen_style6)
        self.pen_style.addStretch(1)
        self.brush_line_type.setLayout(self.pen_style)

    def btnSolidLine(self):
        """
        This method assign the style pen to SolidLine style.

        :return: Nothing
        """
        self.draw.pen_style = Qt.SolidLine

    def btnDashLine(self):
        """
        This method assign the style pen to DashLine style.

        :return: Nothing
        """
        self.draw.pen_style = Qt.DashLine

    def btnDotLine(self):
        """
        This method assign the style pen to DotLine style.

        :return: Nothing
        """
        self.draw.pen_style = Qt.DotLine

    def btndashDotLine(self):
        """
        This method assign the style pen to DashDotLine style.

        :return: Nothing
        """

        self.draw.pen_style = Qt.DashDotLine

    def btnDashDotDotLine(self):
        """
        This method assign the style pen to DashDotDotLine style.

        :return: Nothing
        """

        self.draw.pen_style = Qt.DashDotDotLine

    def btnCustomDashLine(self):
        """
        This method assign the style pen to CustomDashLine style.

        :return: Nothing
        """
        self.draw.pen_style = Qt.CustomDashLine

    def btnwidth1px(self):
        """
        This method assign the style width to 1px.

        :return: Nothing
        """

        self.draw.brushSize = 1

    def btnwidth3px(self):
        """
        This method assign the style width to 3px.

        :return: Nothing
        """
        self.draw.brushSize = 3

    def btnwidth5px(self):
        """
        This method assign the style width to 5px.

        :return: Nothing
        """

        self.draw.brushSize = 5

    def btnwidth8px(self):
        """
        This method assign the style width to 8px.

        :return: Nothing
        """

        self.draw.brushSize = 8

    def btnath(self, b):
        if b.text() == "Curves":
            if b.isChecked():
                self.path_layout_value = 1
        if b.text() == "Lines":
            if b.isChecked():
                self.path_layout_value = 2

    def btnjoin(self, b):
        """
        This method assign the Join style to the pen.
        If bevel is selected, the join style pen will be assigned to BevelJoin style.
        If miter is selected, the join style pen will be assigned to MiterJoin style.
        If round is selected, the join style pen will be assigned to RoundJoin style.

        :return: Nothing
        """

        if b.text() == "Bevel Join":
            if b.isChecked():
                self.draw.pen_join = Qt.BevelJoin
        if b.text() == "Milter Join":
            if b.isChecked():
                self.draw.pen_join = Qt.MilterJoin
        if b.text() == "Round Join":
            if b.isChecked():
                self.draw.pen_join = Qt.RoundJoin

    def btncap(self, b):
        """
        This method assign the Cap style to the pen.
        If flat is selected, the cap style pen will be assigned to FlatCap style.
        If square is selected, the cap style pen will be assigned to SquareCap style.
        If round is selected, the cap style pen will be assigned to RoundCap style.

        :return: Nothing
        """
        if b.text() == "Flat Cap":
            if b.isChecked():
                self.draw.pen_cap = Qt.FlatCap
        if b.text() == "Square Cap":
            if b.isChecked():
                self.draw.pen_cap = Qt.SquareCap
        if b.text() == "Round Cap":
            if b.isChecked():
                self.draw.pen_cap = Qt.RoundCap


class Paint(QDialog):
    """
    This class creates a windows which contains the Paint and the Settings layouts.

    """

    def __init__(self, parent=None):
        """

        The init method declare two classes. First the paint class and then the settings class.
        Both of these classes create a layout.
        Then it's creates a window with a specific size.
        To finish it adds the paint layout at the left and the settings layout at the right of the screen.
        It also set the window title to 'Paint'.
        :param parent: None
        """
        super().__init__(parent)
        self.drawing = Window()
        self.settings = Settings(self.drawing)
        self.setGeometry(800, 600, 1200, 1400)
        grid = QGridLayout()
        grid.addWidget(self.drawing, 0, 1)
        grid.addWidget(self.settings, 0, 2)
        self.setLayout(grid)
        self.setWindowTitle("Paint")


if __name__ == "__main__":
    app = QApplication(sys.argv)
    form = Paint()
    form.show()
    sys.exit(app.exec_())




# Screen Shots of working Project x 2
![Screen Shot1](./screen1.png)
![Screen Shot1](./screen2.png)